In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import re


In [2]:
concerts = pd.read_csv("concerts_clean.csv", index_col=0)
concerts['date'] = pd.to_datetime(concerts['date'], format = "%Y/%m/%d")

In [3]:
concerts.head()

,date,url,venue,artist,location,time
0,2017-02-02,https://www.songkick.com/metro_areas/7644-us-n...,Knitting Factory,Don Giovanni Records Showcase 2017,new-york,NaN
1,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Madison Square Garden,The Lumineers,new-york,NaN
2,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Terminal 5,AFI,new-york,19:00:00-0500
3,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Webster Hall,Keys N Krates (DJ set),new-york,22:00:00-0500
4,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Music Hall of Williamsburg,Bear's Den,new-york,20:00:00-0500


In [4]:
cityview = concerts.groupby('location')


In [5]:
cityview.describe()[0:10]

artist  \
location                                                            
atlanta          count                                        950   
                 unique                                       789   
                 top                                        DJ EU   
                 freq                                          14   
                 first                                        NaN   
                 last                                         NaN   
boston-cambridge count                                        981   
                 unique                                       844   
                 top     Jeremy Lyons: Delta-Silly Music for Kids   
                 freq                                           6   

                                        date           time  \
location                                                      
atlanta          count                   950            682   
                 unique                  142             52   
                 top     2017-02-18 00:00:00  20:00:00-0500   
                 freq                     32             96   
                 first   2017-02-04 00:00:00            NaN   
                 last    2017-11-22 00:00:00            NaN   
boston-cambridge count                   981            772   
                 unique                  148             56   
                 top     2017-02-11 00:00:00  20:00:00-0400   
                 freq                     25            132   

                                                                       url  \
location                                                                     
atlanta          count                                                 950   
                 unique                                                 20   
                 top     https://www.songkick.com/metro_areas/4120-us-a...   
                 freq                                                   50   
                 first                                                 NaN   
                 last                                                  NaN   
boston-cambridge count                                                 981   
                 unique                                                 20   
                 top     https://www.songkick.com/metro_areas/18842-us-...   
                 freq                                                   50   

                                 venue  
location                                
atlanta          count             950  
                 unique            141  
                 top     Eddie's Attic  
                 freq              143  
                 first             NaN  
                 last              NaN  
boston-cambridge count             981  
                 unique            140  
                 top      The Sinclair  
                 freq               71

In [6]:
venueview = concerts.groupby('venue')

In [7]:
def get_venue_artists(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    concert_dates = pd.date_range(start = start_date, end = end_date, freq = 'D')
    artists = []
    for show in concerts.loc[concerts['venue'] == venue, ['date', 'artist']].itertuples():
        if show[1] in concert_dates:
            artists.append(show)
    return artists

In [8]:
#NOTE: I would like to add a way to validate the artist returned by the search here. To do later...
def get_artist_ids(artists):
    sp = spotipy.Spotify()
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist, type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids

In [9]:
def get_venue_artist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    artists = get_venue_artists(venue, start_date, end_date)
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist[2], type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids


In [10]:
def create_venue_songlist(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    songlist = []
    for artist in get_venue_artist_ids(venue, start_date, end_date):
        artist_tracks = sp.artist_top_tracks(artist[1])['tracks']
        if len(artist_tracks) >= 5:
            for track in range(0, 5):
                songlist.append((artist_tracks[track]['name'], \
                                 artist_tracks[track]['id']))
    return songlist
    

In [11]:
def prepare_song_id_list(songlist):
    song_ids = []
    for song in songlist:
        song_ids.append(str(song[1]))
    return song_ids

In [12]:
def create_venue_songlist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    songlist = create_venue_songlist(venue, start_date, end_date)
    print type(songlist)
    if len(songlist) >= 100:
        songlist = songlist[0:99]
    ptitle = ("%s from %s to %s") % (venue, start_date, end_date)
    return (ptitle, prepare_song_id_list(songlist))

In [13]:
####Now that we have a list of song IDs, let's authenticate a spotify login, and create the playlist

In [14]:
SPOTIPY_CLIENT_ID = '1723f7534a1c4cde9efe16b0b2753d7f'
SPOTIPY_CLIENT_SECRET = 'e3e4b35b367a430c8b2900ef5d6165a7'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888'

In [15]:
token = util.prompt_for_user_token("", 'playlist-modify-public', SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI)

In [16]:
import random
def random_venue_tester():
    x = random.choice(range(0, len(concerts['venue'])))
    return concerts['venue'][x]

In [17]:
random_venue_tester()

"Hendershot's"

In [19]:
username = 'afeierman'
playlist_prepped = create_venue_songlist_ids(random_venue_tester(), '2017-04-01', '2017-05-01')
playlist_title = playlist_prepped[0]
track_ids = playlist_prepped[1]

<type 'list'>


In [21]:
print type(token)
print token

<type 'unicode'>
BQBqv-CdBwpLO3Pu3QCFcu4RC5pAnx6--OwzxzVYuYCZiXLo9gR0a9RPNsG68WgB-GziD_0ri4au8jfUnB0ypJP1SDSqJ2T7T4UvKKOYJxiDx1USb9gdV9kVjDW64NdaE-WxyjELBpQvXr4YyCUtWznNOn2cOJosMYP2Hs175CiW4HJcUA_08E1S_6BAKRDhqEfJj-qFE5TNdDyy5zINSWjPPhp1a7hqjX8TTGJ7e8EnCA


In [27]:
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False #Not sure if this is needed
    sp.user_playlist_create(username, playlist_title)
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        if playlist['name'] == playlist_title:
            playlist_id = playlist['id']
            break
    results = sp.user_playlist_add_tracks(username, playlist_id, track_ids)
    print results
else:
    print "Can't get token"

{u'snapshot_id': u'5KGTxiC/a6sbzBgQtHfC/YK1kgpNEvl8VJGdeJ30nN2bPYyIN50ugjOtV/SRjGAe'}
